# Create an Agent for Amazon Bedrock integrated with Knowledge Bases for Amazon Bedrock and attach Action Group

In this notebook you will learn how to create an Amazon Bedrock Agent that makes use of Knowledge Bases for Amazon Bedrock to retrieve company data and complete tasks. The use case for this notebook is an agent for a restaurant, it's tasks will be to give information to the clients about the adults or childrens menu and be in charge of the table booking system. Client's will be able to create, delete or get booking information. 

The steps to complete this notebook are:

1. Import the needed libraries
2. Upload the dataset to Amazon S3
3. Create the Knowledge Base for Amazon Bedrock
4. Create the Agent for Amazon Bedrock
5. Test the Agent
6. Clean-up the resources created

## 1. Import the needed libraries

In [3]:
!pip install -q opensearch-py

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.


In [1]:
import os
import io
import json
import time
import uuid
import boto3
import random
import zipfile
import logging
import pprint
from io import BytesIO
from retrying import retry
from itertools import cycle
from botocore.exceptions import ClientError
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from aoss_utils import createEncryptionPolicy, createNetworkPolicy, createAccessPolicy, createCollection, waitForCollectionCreation
from bedrock_utils import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, delete_iam_role_and_policies

In [79]:
#Clients
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
lambda_client = boto3.client('lambda')
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock = boto3.client("bedrock",region_name=region)
opensearch_client = boto3.client('opensearchserverless')
account_id = sts_client.get_caller_identity()["Account"]
identity_arn = session.client('sts').get_caller_identity()['Arn']
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [80]:
solution_id = "csp-agent-kb-test" ## CREATE A UNIQUE ID FOR YOUR SOLUTION

## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [81]:
bucket_name = solution_id  # Replace with your desired bucket name

if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [82]:
dataset_path = "dataset"
files = [f.name for f in os.scandir(dataset_path) if f.is_file()]
for file in files:
    s3_client.upload_file(f'{dataset_path}/{file}', bucket_name, f'{file}')

## 3. Create a Knowledge Base for Amazon Bedrock

In this section we will go through all the steps to create and test a Knowledge Base. 

These are the steps to complete:
    
1. Create an Amazon Opensearch Serverless Index
2. Define the Embeddings model ARN
3. Create the Knowledge Base
4. Create the data source
5. Sync the Knowledge Base
6. Test the Knowledge Base

### 3.1 Create an Amazon Opensearch Serverless Service Index

To create a new index in Amazon Opensearch Serverless Service you will need to complete the following steps:
- Create an Amazon Bedrock execution role
- Create policies in OpenSearch Service and create the collection
- Define the index settings and mappings and create the index

In [ ]:
def short_uuid():
    uuid_str = str(uuid.uuid4())
    return uuid_str[:8]

short_uuid = short_uuid()

collectionName = "{}-{}".format(solution_id, short_uuid)
indexName = "{}-index-{}".format(solution_id, short_uuid)

print("Collection name:",collectionName)
print("Index name:",indexName)

#### Create an Amazon Bedrock execution role

In [86]:
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

#### Create policies in OpenSearch Service and create the collection

In [ ]:
# Create policies in OpenSearch Service (OSS)
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=collectionName,
    aoss_client=opensearch_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn
)

# Create collection in OpenSearch Service
collection = opensearch_client.create_collection(
    name=collectionName,
    type='VECTORSEARCH'
)

# Wait for collection creation to complete
time.sleep(10)

# Extract collection ID and host
collection_detail = collection.get('createCollectionDetail', {})
collection_id = collection_detail.get('id', '')
host = f"{collection_id}.{region}.aoss.amazonaws.com"

# Print collection details and host
print("Collection:", collection)
print("Host:", host)

# Create OSS policy and attach it to Bedrock execution role
create_oss_policy_attach_bedrock_execution_role(
    collection_id=collection_id,
    bedrock_kb_execution_role=bedrock_kb_execution_role
)
# Wait for all elements to be created
time.sleep(40) 

#### Define the index settings and mappings and create the index

In [ ]:
# Set up AWS authentication
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region, service)

# Define index settings and mappings
index_settings = {
    "settings": {
        "index.knn": "true"
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1536,
                 "method": {
                     "name": "hnsw",
                     "engine": "faiss",
                     "space_type": "innerproduct",
                     "parameters": {
                         "ef_construction": 512,
                         "m": 16
                     },
                 },
             },
            "text": {
                "type": "text"
            },
            "text-metadata": {
                "type": "text"
            }
        }
    }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Create index
response = oss_client.indices.create(index=indexName,body=json.dumps(index_settings))
print(response)

### 3.2 Define the Embedding Model ARN
Define the Embeddings Model ARN which will be used for indexing data into Knowledge Bases for Amazon Bedrock.

In [23]:
embeddingModelArn = "arn:aws:bedrock:{}::foundation-model/amazon.titan-embed-text-v1".format(region)

### 3.3 Create the Knowledge Base
In this section you will create the Knowledge Base, providing the Amazon Bedrock execution role, embeddings model ARN and Opensearch configuration we have previously created. 

In [24]:
knowledge_base_name = solution_id
description = "Example Restaurant KB to combine with Agents for Amazon Bedrock"
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": indexName,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

In [ ]:
@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = knowledge_base_name,
        description = description,
        roleArn = bedrock_kb_execution_role_arn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

try:
    kb = create_knowledge_base_func()
    kb_id = kb["knowledgeBaseId"]
    print(kb)
except Exception as err:
    print(f"{err=}, {type(err)=}")

### 3.4 Create the data source (S3)

After you create your knowledge base, you ingest the data sources into the knowledge base so that they are indexed and able to be queried.

In [ ]:
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}"
}

data_source = bedrock_agent_client.create_data_source(
    knowledgeBaseId=kb_id,
    name='restaurant-menus-source',
    description='Location of the restaurant menus',
    dataSourceConfiguration={
        'type': 'S3',
        's3Configuration': s3Configuration
    },
    vectorIngestionConfiguration={
        'chunkingConfiguration': chunkingStrategyConfiguration
    }
)

data_source_id = data_source["dataSource"]["dataSourceId"]
print("The data source id is: ", data_source_id)

### 3.5 Sync the Knowledge Base
As we have created and associated the data source to the Knowledge Base, we can proceed to Sync the data. 


Each time you add, modify, or remove files from the S3 bucket for a data source, you must sync the data source so that it is re-indexed to the knowledge base. Syncing is incremental, so Amazon Bedrock only processes the objects in your S3 bucket that have been added, modified, or deleted since the last sync.

In [27]:
ingestion_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id,
    dataSourceId=data_source_id,
    description='Initial Ingestion'
)

In [28]:
%%time
status = bedrock_agent_client.get_ingestion_job(
    knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
    dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
    ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
)["ingestionJob"]["status"]
print(status)
while status not in ["COMPLETE", "FAILED", "STOPPED"]:
    status = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
        dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
        ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
    )["ingestionJob"]["status"]
    print(status)
    time.sleep(30)

IN_PROGRESS
IN_PROGRESS
COMPLETE
CPU times: user 10.6 ms, sys: 3.74 ms, total: 14.3 ms
Wall time: 1min


### 3.6 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the **retrieve** and **retrieve_and_generate** functions. 

In [ ]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0".format(region),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
                } 
            },
    retrievalQuery={
        'text': 'Which are the 5 mains available in the childrens menu?'
            
        }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents
# each list has content,location,score,metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

## 4. Create the Agent for Amazon Bedrock

In this section we will go through all the steps to create an Agent for Amazon Bedrock. 

These are the steps to complete:
    
1. Create an Amazon DynamoDB table
2. Create an AWS Lambda function
3. Create the IAM policies needed for the Agent
4. Create the Agent
5. Create the Agent Action Group
6. Allow the Agent to invoke the Action Group Lambda
7. Associate the Knowledge Base to the agent
8. Prepare the Agent and create an alias

### 4.1 Create the DynamoDB table
We will create a DynamoDB table which contains the restaurant bookings information.

In [ ]:
table_name = 'restaurant_bookings'
table = dynamodb_resource.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'booking_id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'booking_id',
            'AttributeType': 'S'
        }
    ],
    BillingMode='PAY_PER_REQUEST'  # Use on-demand capacity mode
)

# Wait for the table to be created
print(f'Creating table {table_name}...')
table.wait_until_exists()
print(f'Table {table_name} created successfully!')

### 4.2 Create the Lambda Function

We will now create a lambda function that interacts with DynamoDB table. To do so we will:

1. Create the `lambda_function.py` file which contains the logic for our lambda function
2. Create the IAM role for our Lambda function
3. Create the lambda function with the required permissions

#### Create the function code

In [ ]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_booking_details(booking_id):
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}

def create_booking(date, hour, num_guests):
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}

def delete_booking(booking_id):
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}

def lambda_handler(event, context):
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = None
        for param in parameters:
            if param["name"] == "booking_id":
                booking_id = param["value"]

        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = None
        hour = None
        num_guests = None
        for param in parameters:
            if param["name"] == "date":
                date = param["value"]
            if param["name"] == "hour":
                hour = param["value"]
            if param["name"] == "num_guests":
                num_guests = int(param["value"])

        if date and hour and num_guests:
            response = str(create_booking(date, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = None
        for param in parameters:
            if param["name"] == "booking_id":
                booking_id = param["value"]

        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

#### Create the required permissions

In [34]:
lambda_function_role = f'{solution_id}-lambda-role'

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

# Attach the AWSLambdaBasicExecutionRole policy
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# Create a policy to grant access to the DynamoDB table
dynamodb_access_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:DeleteItem"
            ],
            "Resource": "arn:aws:dynamodb:{}:{}:table/{}".format(region, account_id, table_name)
        }
    ]
}

# Create the policy
dynamodb_access_policy_json = json.dumps(dynamodb_access_policy)
dynamodb_access_policy_response = iam_client.create_policy(
    PolicyName='{}-DynamoDBAccessPolicy'.format(solution_id),
    PolicyDocument=dynamodb_access_policy_json
)

# Attach the policy to the Lambda function's role
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=dynamodb_access_policy_response['Policy']['Arn']
)

#### Create the function 

In [36]:
lambda_function_name = f'{solution_id}-lambda'

In [37]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=60,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### 4.3 Create the IAM policies needed for the Agent
We first need to create the agent policies that allow bedrock model invocation and Knowledge Base query and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model

In [38]:
agent_bedrock_allow_policy_name = f"{solution_id}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solution_id}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

In [39]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        },
        {
            "Sid": "QueryKB",
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve",
                "bedrock:RetrieveAndGenerate"
            ],
            "Resource": [
                f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
            ]
        },
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

### 4.4 Create the Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [41]:
agent_name = f'{solution_id}-booking-agent'
agent_description = "Agent in charge of a restaurants table bookings"
agent_instruction = "You are a restaurant agent, helping clients retrieve information from their booking, create a new booking or delete an existing booking"

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

In [ ]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

### 4.5 Create the Agent Action Group
We will now create an agent action group that uses the lambda function created before. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In this example, we will provide the Action Group functionality using a `functionSchema`.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [47]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [48]:
agent_action_group_description = "Actions for getting table booking information, create a new booking or delete an existing booking"
agent_action_group_name = "TableBookingsActionGroup"

In [51]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

### 4.6 Allow the Agent to invoke the Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [53]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [ ]:
response

### 4.7 Associate the Knowledge Base to the agent
Now we have created the Agent we can go ahead and associate the Knowledge Base we created earlier. 

In [56]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the plates in the menu.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [ ]:
response

### 4.8 Prepare the Agent and create an alias

Let's create a DRAFT version of the agent that can be used for internal testing.


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

We are also going to create an Agent alias to later on use to invoke it

In [ ]:
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='TestAlias',
    agentId=agent_id,
    description='Test alias',
)

alias_id = response["agentAlias"]["agentAliasId"]
print("The Agent alias is:",alias_id)

## 5. Test the Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [60]:
def invokeAgent(query, session):
    enable_trace:bool = True
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
session_id:str = str(uuid.uuid1())
query = "What are the starters in the childrens menu?"
response = invokeAgent(query, session_id)
print(response)

In [ ]:
query = "I want to create a booking for 2 people, at 8pm on the 5th of May"
response = invokeAgent(query, session_id)
print(response)

In [ ]:
query = "I want to get the information for booking 665f7ca5"
response = invokeAgent(query, session_id)
print(response)

In [ ]:
query = "I want to delete the booking 665f7ca5"
response = invokeAgent(query, session_id)
print(response)

## 6. Clean-up 
Let's delete all the associated resources created to avoid unnecessary costs. 

In [69]:
action_group_id=agent_action_group_response['agentActionGroup']['actionGroupId']

In [71]:
import boto3

def clean_up_resources(
    agent_id, alias_id, lambda_function_name, bucket_name, kb_id, collection_id, agent_role_name, table_name
):

    # Delete Agent Action Group, Agent Alias, and Agent
    try:
        bedrock_agent_client.delete_agent_action_group(agentId=agent_id, agentVersion='DRAFT', actionGroupId=action_group_id)
        bedrock_agent_client.delete_agent_alias(agentAliasId=alias_id, agentId=agent_id)
        bedrock_agent_client.delete_agent(agentId=agent_id)
        print(f"Agent {agent_id}, Agent Alias {alias_id}, and Action Group have been deleted.")
    except Exception as e:
        print(f"Error deleting Agent resources: {e}")

    # Delete Lambda function
    try:
        lambda_client.delete_function(FunctionName=lambda_function_name)
        print(f"Lambda function {lambda_function_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting Lambda function {lambda_function_name}: {e}")

    # Delete all objects in the bucket and the bucket itself
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            for obj in response['Contents']:
                s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} and its objects have been deleted.")
    except Exception as e:
        print(f"Error deleting bucket {bucket_name}: {e}")

    # Delete Knowledge Base
    try:
        bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
        print(f"Knowledge Base {kb_id} has been deleted.")
    except Exception as e:
        print(f"Error deleting Knowledge Base {kb_id}: {e}")

    # Delete Opensearch Collection
    try:
        opensearch_client.delete_collection(id=collection_id)
        print(f"Collection {collection_id} has been deleted.")
    except Exception as e:
        print(f"Error deleting Collection {collection_id}: {e}")

    # Delete Lambda Role and its policies
    try:
        attached_policies = iam_client.list_attached_role_policies(RoleName=agent_role_name)['AttachedPolicies']
        for policy in attached_policies:
            policy_name = policy['PolicyName']
            iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=policy['PolicyArn'])
            iam_client.delete_role_policy(RoleName=agent_role_name, PolicyName=policy_name)
            print(f"Detached and deleted policy {policy_name} from role {agent_role_name}")
        iam_client.delete_role(RoleName=agent_role_name)
        print(f"Role {agent_role_name} has been deleted.")
    except Exception as e:
        pass
        print(f"Error deleting role {agent_role_name}: {e}")

    # Delete DynamoDB table
    try:
        dynamodb_client.delete_table(TableName=table_name)
        print(f"Table {table_name} is being deleted...")
        waiter = dynamodb_client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(f"Table {table_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {e}")

In [ ]:
clean_up_resources(agent_id, alias_id, lambda_function_name, bucket_name, kb_id, collection_id, agent_role_name, table_name)
delete_iam_role_and_policies()